In [ ]:
# import os
# import glob
# import pandas as pd

# nodeidx2asin = pd.read_csv(
#     'dataset/ogbn_products/mapping/nodeidx2asin.csv.gz', compression='gzip')
# nodeidx2asin.set_index('asin', inplace=True)


# responses = glob.glob('output/ogbn-products/*.json')

# for res in responses:
#     asin = res.split('/')[-1].split('.json')[0]
#     nodeidx = nodeidx2asin.loc[asin]['node idx']
#     fn = f'output/ogbn-products/{nodeidx}.json'
#     os.rename(res, fn)

In [ ]:
import json
import re
import numpy as np
import glob
import pandas as pd

from ogb.nodeproppred import PygNodePropPredDataset
import torch_geometric.transforms as T


dataset = PygNodePropPredDataset(
    name='ogbn-products', transform=T.ToSparseTensor())
data = dataset[0]

responses = glob.glob('/home/xiaoxin/llama/output/ogbn-products/*.json')
print(len(responses))

# classes = labelidx2productcategory['product category'].str.lower().tolist()
classes = ['home & kitchen', 'health & personal care', 'beauty', 'sports & outdoors', 'books', 'patio, lawn & garden', 'toys & games', 'cds & vinyl', 'cell phones & accessories', 'grocery & gourmet food', 'arts, crafts & sewing', 'clothing, shoes & jewelry', 'electronics', 'movies & tv', 'software', 'video games', 'automotive', 'pet supplies', 'office products', 'industrial & scientific', 'musical instruments', 'tools & home improvement',
           'magazine subscriptions', 'baby products', 'nan', 'appliances', 'kitchen & dining', 'collectibles & fine art', 'all beauty', 'luxury beauty', 'amazon fashion', 'computers', 'all electronics', 'purchase circles', 'mp3 players & accessories', 'gift cards', 'office & school supplies', 'home improvement', 'camera & photo', 'gps & navigation', 'digital music', 'car electronics', 'baby', 'kindle store', 'kindle apps', 'furniture & decor']
classes
classes = [i if i is not np.nan else 'nan' for i in classes]
classes_regex = '(' + '|'.join(classes) + ')'
class_map = {x: i for i, x in enumerate(classes)}


pred = []
limit = len(responses)
for fn in responses:
    response = json.load(open(fn, "r"))
    answer = response['generation']['content'].lower()

    matches = re.findall(classes_regex, answer.strip())

    mapped = [class_map[m] for m in matches]

    if len(mapped) == 0:
        # print("EMPTY: ", answer)
        mapped = [1]

    pred.append(mapped)

responses_nodeidx = [int(res.split('/')[-1].split('.json')[0])
                     for res in responses]
labels = data.y[responses_nodeidx]

accs = []
for k in range(1, 6):
    acc = np.mean([labels[row].item() in pred[row][:k]
                  for row in range(limit)])
    accs.append(acc)
print('accs: ', accs)
print('done')

In [7]:
import numpy as np
import re
import pandas as pd
import json
import glob
from utils.load_arxiv import get_raw_text_arxiv

data, text = get_raw_text_arxiv(use_text=False)


label2text = pd.read_csv(
    "./dataset/{}/label2text.csv".format('ogbn_arxiv'))
options = label2text['arxiv category'].str.slice(start=9).str.upper().tolist()
pred = []

responses = glob.glob('output/ogbn-arxiv/*.json')
answers = []
idx_to_query = []
for res in responses:
    idx_to_query.append(int(res.split('/')[-1].split('.json')[0]))
    response = json.load(open(res))
    answer = response['generation']['content']
    matches = re.findall(r"cs\.[A-Z]{2}", answer.strip())
    topk = [x[3:] for x in list(dict.fromkeys(matches))[:5]]
    pred.append([options.index(c) for c in topk if c in options])


# compute top-k accuracy for k=1,...5
labels = data.y[idx_to_query]
accs = []
for k in range(1, 6):
    acc = np.mean([labels[row].item() in pred[row][:k]
                  for row in range(len(idx_to_query))])
    accs.append(acc)

print(f"# of samples: {len(idx_to_query)}")
print('accs: ', accs)
print('done')

# of samples: 2242
accs:  [0.4023193577163247, 0.5695807314897413, 0.6271186440677966, 0.6578947368421053, 0.6623550401427297]
done
